In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from os import listdir

In [43]:
# taxfiles = listdir('../../MOTUS_TAX/')
# tax_values = []; samples = [];
# for f in taxfiles:
#     temp = pd.read_csv('../../MOTUS_TAX/'+f,skiprows=2,sep='\t')
#     tax_values.append(list(temp['unnamed sample']))
#     samples.append(f.split('_')[0])
# temp = pd.read_csv('../../MOTUS_TAX/A10_tax.txt',skiprows=2,sep='\t')
# taxnames = [ x.split('[')[0].strip() for x in list(temp['#consensus_taxonomy'])]


# oral_tax = pd.DataFrame(zip(taxnames),columns=['Species'])
# for i in range(len(samples)):
#     oral_tax[samples[i]]=tax_values[i]
    
# oral_tax = oral_tax.set_index(['Species'])
# oral_tax = oral_tax.loc[(oral_tax !=0).any(axis=1)]
# oral_tax.to_csv('../data/metagenome/motus_profile.csv')